In [9]:
import pandas as pd

In [10]:
class Fase:
    def __init__(self, tickets, puntos, horquilla=0, oso=0, tacon=0, s_escudo=0, s_espada=0, s_porra=0, s_bombardero=0, s_mago=0):
        self.tickets = tickets
        self.puntos = puntos
        
        self.objetos = {
            "horquillas": horquilla,
            "osos": oso,
            "tacones": tacon,
            "soldados_escudo": s_escudo,
            "soldados_espada": s_espada,
            "soldados_porra": s_porra,
            "soldados_bombardero": s_bombardero,
            "soldados_mago": s_mago
        }

# Definir los valores para 10 fases diferentes
fase1 = Fase(10, 30, horquilla=60, oso=20, s_escudo=1, s_espada=1)
fase2 = Fase(10, 30, oso=50, tacon=20, s_porra=1, s_bombardero=1)
fase3 = Fase(10, 30, horquilla=20, tacon=50, s_mago=1)
fase4 = Fase(12, 40, horquilla=160, s_espada=1, s_escudo=1)
fase5 = Fase(12, 40, oso=130, s_porra=1, s_bombardero=1)
fase6 = Fase(12, 40, tacon=110, s_mago=1) 
fase7 = Fase(15, 45, horquilla=300, s_escudo=1, s_espada=1)
fase8 = Fase(15, 45, oso=240, s_porra=1, s_bombardero=1)
fase9 = Fase(15, 45, tacon=200, s_mago=1)

fases = (fase1, fase2, fase3, fase4, fase5, fase6, fase7, fase8, fase9, fase10)

# Bonus actuales
b_punt = 1.05
b_hor = 1.40
b_oso = 1.15
b_taco = 1.1

for fase in fases:
    fase.puntos = int(fase.puntos * b_punt)
    fase.objetos["horquillas"] = int(fase.objetos["horquillas"] * b_hor)
    fase.objetos["osos"] = int(fase.objetos["osos"] * b_oso)
    fase.objetos["tacones"] = int(fase.objetos["tacones"] * b_taco)


In [11]:
def inicializar_puntos(path, puntos=0, horquilla=0, oso=0, tacon=0, s_escudo=0, s_espada=0, s_porra=0, s_bombardero=0, s_mago=0):
    data = {
    "Tipo": ("puntos", "horquillas", "osos", "tacones", "soldados_escudo", "soldados_espada", "soldados_porra", "soldados_bombardero", "soldados_mago"),
    "Puntos_Actuales": (puntos, horquilla, oso, tacon, s_escudo, s_espada, s_porra, s_bombardero, s_mago),
    "Puntos_Maximos" : (10000, 10000, 10000, 10000, 50, 50, 50, 50, 50)
    }
    
    df = pd.DataFrame(data).set_index("Tipo")
    df["Puntos_Restantes"] = df.Puntos_Maximos - df.Puntos_Actuales 
    df.to_json(path, indent=4)
    print("Datos inicializados")
    
    return df

In [12]:
def read_df(path : str):
    try:
        df = pd.read_json(path)
        return df
    except:
        print("No se ha encontrado el archivo")
        return False

In [13]:
def print_info(tickets : int, df : pd.DataFrame, fases : tuple):
    
    print(df)

    print(f"\n{tickets} tickets disponibles\n")
    i = 1
    
    for fase in fases:
        rondas = int(tickets/fase.tickets)
        
        if rondas != 0:
            print(f"Fase {i} ({rondas} rondas): {fase.puntos* rondas} puntos \n")
            
            for nombre, valor in fase.objetos.items():
                if valor != 0:
                    print(f"{valor * rondas} {nombre}")
            print("\n")
        i += 1
        

In [14]:
def actualizar_puntos(fase : int, rondas : int, fases : tuple, df : pd.DataFrame, path : str):
    if rondas > 10 or rondas <= 0:
        print("No se puede realizar ese numero de rondas")
        return df
    else:
        fase -= 1
        df.Puntos_Actuales["puntos"] += fases[fase].puntos * rondas
        for nombre in fases[fase].objetos.keys():
            df.Puntos_Actuales[nombre] += fases[fase].objetos[nombre] * rondas
        
        df["Puntos_Restantes"] = df.Puntos_Maximos - df.Puntos_Actuales
        df.to_json(path, indent=4)
        print("Actualizacion completada")
            
    return df

In [15]:
path = "puntos.json"
df = read_df(path)

decision = input("Que quieres hacer: 1 = comprobar puntos, 2 = aplicar puntos, 3 = inicializar datos, otros = salir")
try: 
    decision = int(decision)
except ValueError:
    pass

match decision:
    case 1:
        tickets = input("Tickets disponibles")
        try:  
            tickets = int(tickets)
            if 180 >= tickets > 0:
                print_info(tickets, df, fases)
            else:
                print("No es una cantidad permitida")
        except ValueError: 
            print("No son numeros enteros")
                 
    case 2: 
        fase, rondas = input("Que fase has completado y cuantas rondas, separado por un espacio").split(" ")
        
        try:
            fase = int(fase)
            rondas = int(rondas)
            actualizar_puntos(fase, rondas, fases, df, path)
        except ValueError:
            print("La fase y/o las veces completadas no son numeros enteros")
    
    case 3:
        decision2 = input("Que quieres hacer: 1 = introducir datos manualmente, 2 = inicializar de cero, otros = salir")
        try: 
            decision2 = int(decision2)
        except ValueError:
            pass
        
        match decision2:
            case 1:
                df = read_df(path)
                if df is False:
                    inicializar_puntos(path)
                else:
                    continuar = True
                    while continuar:
                        data = input("Escribe separados por espacios los valores iniciales de: puntos, horquillas, osos, tacones, soldados_escudo, soldados_espada, soldados_porra, soldados_bombardero, soldados_mago. Para salir pulsa solo enter.").split(" ")
                        if data == [""]:
                            continuar = False
                            print("Cancelando operacion")
                        elif len(data) == 9:
                            fallo = False
                            for i in data:
                                try: 
                                    data[data.index(i)] = int(i)
                                except ValueError:
                                    print("No son numeros enteros")
                                    fallo = True
                                    break
                            if not fallo:
                                df = inicializar_puntos(path, data[0], data[1], data[2], data[3], data[4], data[5], data[6], data[7], data[8])
                                continuar = False
                        else:
                            print("No se han introducido los datos correctos")
            case 2:
                df = read_df(path)
                if df is False:
                    inicializar_puntos(path)
                else:
                    continuar = True
                    while continuar:
                        validacion = input("Estas seguro que quieres reiniciar los datos? Y/N")
                        if validacion == "Y" or validacion == "y":
                            inicializar_puntos(path)
                            continuar = False
                        elif validacion == "N" or validacion == "n":
                            continuar = False
                            print("Cancelando operacion")
                        else:
                            print("No es una respuesta valida")
            case other:
                print("adios")
        
    case other:
        print("Adios")

Datos inicializados


Prueba de class vs json

In [16]:
def puntos_fases_json(path = "fases.json"):
    df = read_df("fases.json")
    b_punt = 1
    b_hor = 1.40
    b_oso = 1.15
    b_taco = 1.1

    df["bonus"] = [1, b_punt, b_hor, b_oso, b_taco, 1,1,1,1,1]
    for col in df.columns:
        if col != "bonus":
            name = f"{col}_bonus"
            df[name] = df[col] * df["bonus"]
            # df.astype({name : "int"})
            df = df.drop(col, axis=1)
    df = df.drop("bonus", axis=1)

    return df

df_fases = puntos_fases_json()
print(df_fases.isnull().count())
df_fases

fase_1_bonus    10
fase_2_bonus    10
fase_3_bonus    10
fase_4_bonus    10
fase_5_bonus    10
fase_6_bonus    10
fase_7_bonus    10
fase_8_bonus    10
fase_9_bonus    10
dtype: int64


,fase_1_bonus,fase_2_bonus,fase_3_bonus,fase_4_bonus,fase_5_bonus,fase_6_bonus,fase_7_bonus,fase_8_bonus,fase_9_bonus
tickets,10.0,10.0,10.0,12.0,12.0,12.0,15.0,15.0,15.0
puntos,30.0,30.0,30.0,40.0,40.0,40.0,45.0,45.0,45.0
horquillas,84.0,0.0,28.0,224.0,0.0,0.0,420.0,0.0,0.0
osos,23.0,57.5,0.0,0.0,149.5,0.0,0.0,276.0,0.0
tacones,0.0,22.0,55.0,0.0,0.0,121.0,0.0,0.0,220.0
soldados_escudo,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
soldados_espada,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
soldados_porra,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
soldados_bombardero,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
soldados_mago,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
